# Handling missing data and outliers

Data cleaning is one of the first steps in any Data Science project. It includes filtering and modifying data to make it easier to explore, understand and model.

## Table of Contents
1. [Missing data](#1.-Missing-data)
2. [Outliers](#2.-Outliers)

# 1. Missing data
A few ways to handle missing data in pandas:

- Check for NANs: pd.isnull(object)
- Drop missing data: df.dropna(axis=0, how='any')
- Replace missing data: df.replace(to_replace=None, value=None)
- Drop a feature: df.drop('feature_variable_name', axis=1)

### Import data

In [1]:
import pandas as pd

vdotDf = pd.read_csv('./datasets/VDOT_Traffic_Volume.csv')
vdotDf.head(5)

,OBJECTID,DATA_DATE,ROUTE_COMMON_NAME,START_LABEL,END_LABEL,ADT,ADT_QUALITY,PERCENT_4_TIRE,PERCENT_BUS,PERCENT_TRUCK_2_AXLE,...,CLASS_QUALITY_CODE,AAWDT,AAWDT_QUALITY_CODE,FROM_JURISDICTION,TO_JURISDICTION,ROUTE_NAME,FROM_DISTRICT,TO_DISTRICT,RTE_TYPE_CD,Shape__Length
0,1,2011-08-03T00:00:00.000Z,SC-2901N (Accomack County),Bus US 13,Dead End,100.0,R,NaN,NaN,NaN,...,X,NaN,X,Accomack County,Accomack County,R-VA001SC02901NB,Hampton Roads,Hampton Roads,SC,253.337714
1,2,2013-05-15T00:00:00.000Z,SC-1383N (Prince William County),Cul-de-Sac,76-1279 Longview Dr,60.0,M,NaN,NaN,NaN,...,X,NaN,X,Prince William County,Prince William County,R-VA076SC01383NB,Northern Virginia,Northern Virginia,SC,111.903491
2,3,2014-08-05T00:00:00.000Z,SC-2352N (Hanover County),42-1685 Daffodil Rd,42-2351 Sydnor Lane,100.0,R,NaN,NaN,NaN,...,X,NaN,X,Hanover County,Hanover County,R-VA042SC02352NB,Richmond,Richmond,SC,582.523332
3,4,2013-04-04T00:00:00.000Z,SC-932N (Augusta County),Dead End,0.11 MN Dead End,5.0,R,NaN,NaN,NaN,...,X,NaN,X,Augusta County,Augusta County,R-VA007SC00932NB,Staunton,Staunton,SC,288.684324
4,5,2005-06-13T00:00:00.000Z,SC-1331E (Dinwiddie County),26-1325 Creek Ave,Cul-de-Sac,40.0,R,NaN,NaN,NaN,...,X,NaN,X,Dinwiddie County,Dinwiddie County,R-VA026SC01331EB,Richmond,Richmond,SC,58.437794


In [2]:
vdotDf.shape

(121811, 23)

### Check for missing data

In [3]:
missing_data = vdotDf.isnull().sum()
missing_data

OBJECTID                     0
DATA_DATE                    0
ROUTE_COMMON_NAME            0
START_LABEL                  0
END_LABEL                    0
ADT                          8
ADT_QUALITY                  0
PERCENT_4_TIRE           96354
PERCENT_BUS              96354
PERCENT_TRUCK_2_AXLE     96354
PERCENT_TRUCK_3_AXLE     96354
PERCENT_TRUCK_1_TRAIL    96354
PERCENT_TRUCK_2_TRAIL    96354
CLASS_QUALITY_CODE           0
AAWDT                    94126
AAWDT_QUALITY_CODE          15
FROM_JURISDICTION           97
TO_JURISDICTION            165
ROUTE_NAME                   0
FROM_DISTRICT              812
TO_DISTRICT                850
RTE_TYPE_CD                  0
Shape__Length              575
dtype: int64

### Missing data percentage

In [4]:
import numpy as np
(missing_data.sum()/np.product(vdotDf.shape))*100

24.08478137727977

### Drop rows
- Delete a row if the column value is null or NaN
- Warning: `dropna()` can end up deleting important row/column

In [5]:
del_cols = ['PERCENT_4_TIRE', 'PERCENT_BUS', 'PERCENT_TRUCK_2_AXLE', 'PERCENT_TRUCK_3_AXLE', 'PERCENT_TRUCK_1_TRAIL','PERCENT_TRUCK_2_TRAIL']
vdotDf = vdotDf.dropna(subset=del_cols)
vdotDf.shape

(25457, 23)

### Replace missing data
- Missing data can be replaced by mean, median or mode
- Warning: This is just an example. Replacing missing data with mean value is **not** ideal in this case

In [6]:
vdotDf['Shape__Length'].fillna((vdotDf['Shape__Length']).mean(), inplace=True)

### After dropping and replacing rows

In [7]:
missing_data = vdotDf.isnull().sum()
missing_data

OBJECTID                   0
DATA_DATE                  0
ROUTE_COMMON_NAME          0
START_LABEL                0
END_LABEL                  0
ADT                        0
ADT_QUALITY                0
PERCENT_4_TIRE             0
PERCENT_BUS                0
PERCENT_TRUCK_2_AXLE       0
PERCENT_TRUCK_3_AXLE       0
PERCENT_TRUCK_1_TRAIL      0
PERCENT_TRUCK_2_TRAIL      0
CLASS_QUALITY_CODE         0
AAWDT                     19
AAWDT_QUALITY_CODE         0
FROM_JURISDICTION         25
TO_JURISDICTION           58
ROUTE_NAME                 0
FROM_DISTRICT            462
TO_DISTRICT              484
RTE_TYPE_CD                0
Shape__Length              0
dtype: int64

# 2. Outliers
Outliers in a dataset can be found using z-score. 

Z-score is the number of standard deviations from the mean a data point is. It ranges from -3 standard deviations (which would fall to the far left of the normal distribution curve) up to +3 standard deviations.

### List columns with numeric values

In [8]:
# Select numeric columns
numericVdotDf = vdotDf.select_dtypes(include=[np.number])
numericVdotDf.dtypes

OBJECTID                   int64
ADT                      float64
PERCENT_4_TIRE           float64
PERCENT_BUS              float64
PERCENT_TRUCK_2_AXLE     float64
PERCENT_TRUCK_3_AXLE     float64
PERCENT_TRUCK_1_TRAIL    float64
PERCENT_TRUCK_2_TRAIL    float64
AAWDT                    float64
Shape__Length            float64
dtype: object

### Basic statistical details

In [9]:
numericVdotDf['Shape__Length'].describe()

count    25457.000000
mean      2360.722389
std       2790.669661
min          0.121676
25%        537.514130
50%       1355.110645
75%       3039.613126
max      27341.396063
Name: Shape__Length, dtype: float64

### Calculate z scores

In [10]:
# import stats from scipy library
from scipy import stats

z = np.abs(stats.zscore(numericVdotDf['Shape__Length']))
print(z)

[0.73605406 0.42562468 0.4300108  ... 0.80225336 0.79165523 2.00697132]


### Find rows where z-score is greater than 3

In [11]:
# Location (row index) where Shape__Length column has z_score greater than 3
outliers_loc = np.where(z > 3)
outliers_loc

(array([   93,   120,   126,   128,   134,   139,   144,   218,   430,
          437,   462,   499,   858,   874,   877,   882,   896,   913,
          917,   922,   942,   961,   995,  1142,  1284,  1424,  1455,
         1485,  1510,  1586,  1721,  1763,  2001,  2102,  2110,  2111,
         2118,  2129,  2136,  2137,  2201,  2211,  2231,  2340,  2456,
         2479,  2503,  2506,  2577,  2583,  2585,  2589,  2596,  2598,
         2600,  2602,  2680,  2685,  2761,  2780,  2782,  3027,  3248,
         3300,  3376,  3383,  3408,  3416,  3508,  3552,  3772,  3848,
         3850,  3973,  4068,  4162,  4173,  4175,  4176,  4182,  4196,
         4208,  4213,  4224,  4225,  4233,  4238,  4248,  4249,  4300,
         4312,  4319,  4320,  4442,  4446,  4456,  4462,  4489,  4519,
         4965,  4973,  4983,  4985,  5175,  5177,  5196,  5253,  5299,
         5303,  5307,  5317,  5324,  5365,  5447,  5717,  5719,  5723,
         5764,  5770,  5773,  5779,  5806,  5820,  5845,  5978,  6097,
      

In [12]:
# Find column index of Shape__Length
numericVdotDf.columns.get_loc("Shape__Length")

9

In [13]:
# Display 5 of the outlier values
numericVdotDf.iloc[[93, 120, 126, 128, 134], 9]

998     11418.144600
1859    11015.357792
1886    12532.867202
1892    14159.382355
1903    15005.123606
Name: Shape__Length, dtype: float64